In [1]:
# create spark context
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType
from pyspark.sql.functions import month, desc

spark = SparkSession.builder.getOrCreate()

In [2]:
spark

In [3]:
# specify schema for the crime dataset
schema = (StructType([
    StructField("X", FloatType(), True),
    StructField("Y", FloatType(), True),
    StructField("RowID", IntegerType(), True),
    StructField("CrimeDateTime", TimestampType(), True),
    StructField("CrimeCode", StringType(), True),
    StructField("Location", StringType(), True),
    StructField("Description", StringType(), True),
    StructField("Inside_Outside", StringType(), True),
    StructField("Weapon", StringType(), True),
    StructField("Post", IntegerType(), True),
    StructField("District", StringType(), True),
    StructField("Neighborhood", StringType(), True),
    StructField("Latitude", FloatType(), True),
    StructField("Longitude", FloatType(), True),
    StructField("GeoLocation", StringType(), True),
    StructField("Premise", StringType(), True),
    StructField("VRIName", StringType(), True),
    StructField("Total_Incidents", IntegerType(), True)
]))

# read the file using the schema definition
df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("delimiter", ",") \
    .option("timestampFormat", "yyyy/MM/dd HH:mm:ss+00") \
    .schema(schema) \
    .load('Part1_Crime_data.csv')

In [4]:
# cache the dataframe
df = df.persist()

In [5]:
# show the count of rows
df.count()

513158

In [6]:
# print the schema
df.printSchema()

root
 |-- X: float (nullable = true)
 |-- Y: float (nullable = true)
 |-- RowID: integer (nullable = true)
 |-- CrimeDateTime: timestamp (nullable = true)
 |-- CrimeCode: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Inside_Outside: string (nullable = true)
 |-- Weapon: string (nullable = true)
 |-- Post: integer (nullable = true)
 |-- District: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Latitude: float (nullable = true)
 |-- Longitude: float (nullable = true)
 |-- GeoLocation: string (nullable = true)
 |-- Premise: string (nullable = true)
 |-- VRIName: string (nullable = true)
 |-- Total_Incidents: integer (nullable = true)



In [7]:
# show first 5 rows
df.show(5)

+---------+---------+-----+-------------------+---------+-------------------+--------------+--------------+-------+----+---------+----------------+--------+---------+------------------+---------------+------------+---------------+
|        X|        Y|RowID|      CrimeDateTime|CrimeCode|           Location|   Description|Inside_Outside| Weapon|Post| District|    Neighborhood|Latitude|Longitude|       GeoLocation|        Premise|     VRIName|Total_Incidents|
+---------+---------+-----+-------------------+---------+-------------------+--------------+--------------+-------+----+---------+----------------+--------+---------+------------------+---------------+------------+---------------+
|1420074.1| 594160.6|    1|2022-02-26 04:00:00|       4E|  200 W MONUMENT ST|COMMON ASSAULT|          null|     NA| 124|  CENTRAL|    MOUNT VERNON| 39.2975| -76.6193|(39.2975,-76.6193)|           null|        null|              1|
|1411374.2| 589791.4|    2|2022-02-26 01:26:30|       9S| 2100 FREDERICK AVE

In [8]:
# What are distinct crime codes?
df.select('CrimeCode').distinct().show(100)

+---------+
|CrimeCode|
+---------+
|       3P|
|       3K|
|      3BJ|
|       1A|
|       3M|
|       5F|
|       4B|
|       3B|
|       7A|
|      3NF|
|      3EF|
|       3N|
|       5D|
|      3LO|
|      3AF|
|       7B|
|      3GO|
|     3AJF|
|      8AO|
|       7C|
|      3AK|
|      3GK|
|       6L|
|      3EO|
|      3JO|
|       3F|
|       1K|
|       8H|
|       4C|
|       5A|
|       6C|
|      3NK|
|       3D|
|       6H|
|      3LK|
|     3AJK|
|      3CO|
|       3L|
|       4E|
|       6D|
|       2A|
|       3C|
|       8I|
|      3NO|
|      3JF|
|       3E|
|      3LF|
|       1O|
|       8J|
|      3CK|
|      8BO|
|       2B|
|      3JK|
|       5B|
|       4A|
|      8GO|
|      8EV|
|      3CF|
|      8EO|
|       6G|
|       6A|
|       9S|
|      3EK|
|      3GF|
|       3H|
|       4D|
|      8FO|
|       6J|
|       6F|
|       6E|
|       3J|
|       5C|
|      8AV|
|       5E|
|     3AJO|
|      3AO|
|       6B|
|      8CV|
|      8BV|
|      8FV|
|   

In [9]:
# Count the number of crimes by the crime codes and order by the resulting counts in descending order
# top 20
df_sorted = df.groupBy("CrimeCode").count().sort(desc("Count"))
df_sorted.show()

+---------+-----+
|CrimeCode|count|
+---------+-----+
|       4E|91650|
|       6D|68427|
|       5A|43928|
|       7A|40274|
|       6J|27636|
|       6G|26858|
|       6E|24300|
|       6C|23227|
|       4C|22438|
|       5D|14971|
|      3AF|14739|
|       4B|14460|
|       4A|13226|
|       3B|10737|
|       4D| 7232|
|       5B| 6475|
|       9S| 5443|
|       6F| 5081|
|       5C| 4917|
|       6B| 4106|
+---------+-----+
only showing top 20 rows



In [10]:
# Which neighborhood had most crimes?
df_neighborhood = df.groupBy("Neighborhood").count().sort(desc("count"))
df_neighborhood.show(1)

+------------+-----+
|Neighborhood|count|
+------------+-----+
|    DOWNTOWN|17799|
+------------+-----+
only showing top 1 row



In [11]:
# Which month of the year had most crimes?
df.groupBy(month("CrimeDateTime").alias("Month")).count().sort(desc("count")).show(1)

+-----+-----+
|Month|count|
+-----+-----+
|    8|46327|
+-----+-----+
only showing top 1 row



In [27]:
# What weapons were used? Exclude nulls - store as new
df_weapons = df.filter((df.Weapon != "NA") & (df.Weapon.isNotNull())).cache()
df_weapons.select('Weapon').distinct().show()

+-------+
| Weapon|
+-------+
|  HANDS|
|  KNIFE|
|  OTHER|
|   FIRE|
|FIREARM|
+-------+



In [29]:
# Which weapon was used the most?
df_weapons.groupBy("Weapon").count().sort(desc("count")).show(1)

+-------+-----+
| Weapon|count|
+-------+-----+
|FIREARM|46139|
+-------+-----+
only showing top 1 row

